In [1]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm
import pandas as pd

2023-04-17 15:26:46.757844: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 15:26:46.887523: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-17 15:26:47.535662: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dnlab/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-04-17 15:26:47.

In [2]:
class_names = ['driving', 'falling_asleep', 'using_cellphone']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (150, 150)

print(class_names_label)
print(nb_classes)

{'driving': 0, 'falling_asleep': 1, 'using_cellphone': 2}
3


In [3]:
# Model with Augmented image sets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_data_aug():
    datasets = ['/home/dnlab/Data-B/data/main_data/train_cat_new', '/home/dnlab/Data-B/data/main_data/val_cat_new']
    output = []

    # Define image augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    val_datagen = ImageDataGenerator(rescale=1./255)

    for dataset in datasets:
        images = []
        labels = []

        print('Loading {}'.format(dataset))
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            # iter through each imag in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):

                # get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)

                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE)

                # Apply data augmentation
                if dataset == '/home/dnlab/Data-B/data/main_data/train_cat_new':
                    augmented_image = train_datagen.random_transform(image)
                else:
                    augmented_image = val_datagen.random_transform(image)

                images.append(augmented_image)
                labels.append(label)

        images = np.array(images, dtype='float32')
        labels = np.array(labels, dtype='int32')

        output.append((images, labels))

    return output


In [4]:
(train_images2, train_labels2), (test_images2, test_labels2) = load_data_aug()

Loading /home/dnlab/Data-B/data/main_data/train_cat_new


100%|██████████| 2664/2664 [00:12<00:00, 211.70it/s]


Loading /home/dnlab/Data-B/data/main_data/val_cat_new


100%|██████████| 320/320 [00:00<00:00, 1399.74it/s]


In [5]:

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_images2, train_labels2, batch_size=8, epochs=20, validation_split = 0.2)

2023-04-17 15:27:32.518466: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 80 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:73:00.0, compute capability: 8.6
2023-04-17 15:27:43.932360: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.73GiB (rounded to 1856520192)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-04-17 15:27:43.932449: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-04-17 15:27:43.932483: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 17, Chunks in use: 17. 4.2KiB allocated for chunks. 4.2KiB in use in bin. 344B client-requested in use in bin.
2023-04-17 15:

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
import matplotlib.pyplot as plt

def plot_accuracy_loss(history):
    # Plot loss
    fig, ax = plt.subplots(figsize=(8, 6))
    plt.plot(history.history['loss'],'bo--', label = "train_loss")
    plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.legend()
    
    # Plot accuracy
    fig, ax = plt.subplots(figsize=(8, 6))
    plt.plot(history.history['accuracy'],'bo--', label = "train_acc")
    plt.plot(history.history['val_accuracy'], 'ro--', label = "val_acc")
    plt.title("train_acc vs val_acc")
    
    plt.legend()
    plt.show()

# Plot accuracy and loss graphs
plot_accuracy_loss(history)


In [ ]:
def display_examples(class_names, images, true_labels, predicted_labels=None):
    """
    Display 25 images from the images array with their corresponding labels and predicted labels
    """
    fig = plt.figure(figsize=(10, 10))
    fig.suptitle("Some examples of images of the dataset", fontsize=16)
    for i in range(25):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        if predicted_labels is None:
            plt.xlabel(class_names[true_labels[i]])
        else:
            true_label = class_names[true_labels[i]]
            predicted_label = class_names[np.argmax(predicted_labels[i])]
            if true_label == predicted_label:
                color = 'green'
            else:
                color = 'red'
            plt.xlabel(f"True:{true_label}\nPred:{predicted_label}", color=color, fontsize=8)
    plt.show()


In [ ]:
import numpy as np

train_images = train_images2 / 255.0 
test_images = test_images2 / 255.0


# Assuming you have a trained model called "model"
predicted_labels = model.predict(test_images)

# Select 25 random images and their corresponding labels
indices = np.random.choice(len(test_images), size=25, replace=False)
images_to_plot = test_images[indices]
true_labels = test_labels2[indices]
predicted_labels = predicted_labels[indices]

# Display the selected images with their true and predicted labels
display_examples(class_names, images_to_plot, true_labels, predicted_labels)